<a href="https://colab.research.google.com/github/Pradnya1208/Intent-classification-based-food-delivery-chatbot/blob/main/Chatbot%20implementation/Initial_Chatbot_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [ ]:
with open('/content/intents.json') as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [ ]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [ ]:
vocab_size = 11250
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=None)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

In [142]:

epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/500
12/12 [==============================] - 1s 17ms/step - loss: 3.0907 - accuracy: 0.0275
Epoch 2/500
12/12 [==============================] - 0s 13ms/step - loss: 3.0849 - accuracy: 0.1291
Epoch 3/500
12/12 [==============================] - 0s 14ms/step - loss: 3.0795 - accuracy: 0.1319
Epoch 4/500
12/12 [==============================] - 0s 14ms/step - loss: 3.0736 - accuracy: 0.1319
Epoch 5/500
12/12 [==============================] - 0s 18ms/step - loss: 3.0663 - accuracy: 0.1319
Epoch 6/500
12/12 [==============================] - 0s 24ms/step - loss: 3.0589 - accuracy: 0.1319
Epoch 7/500
12/12 [==============================] - 0s 24ms/step - loss: 3.0479 - accuracy: 0.1319
Epoch 8/500
12/12 [==============================] - 0s 14ms/step - loss: 3.0348 - accuracy: 0.1319
Epoch 9/500
12/12 [==============================] - 0s 19ms/step - loss: 3.0193 - accuracy: 0.1319
Epoch 10/500
12/12 [==============================] - 0s 23ms/step - loss: 2.9996 - accuracy: 0.1319

In [143]:
#tokenizer.oov_token = None 

In [145]:
# saving model
model.save("chat_model")

import pickle

# saving tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

# saving label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model/assets


INFO:tensorflow:Assets written to: chat_model/assets


In [146]:
!pip install colorama

In [148]:
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("/content/intents.json") as file:
    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])
        print(tag)

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()


Start messaging with the bot (type quit to stop)!
User: hi
['greeting']
ChatBot: Hey! I'm Restrobot. How can I help you?
User: want to order a food
['general']
ChatBot: :)
User: veg food
['veg_enquiry']
ChatBot: 
User: need some food
['Bread_options']
ChatBot: 
User: i am hungry
['menu']
ChatBot: 
User: what are the options in bread
['Bread_options']
ChatBot: 
User: menu please
['menu']
ChatBot: 
User: some offers
['offers']
ChatBot: 
User: what's special today?
['veg_enquiry']
ChatBot: 
User: chicken dinner
['nonveg_enquiry']
ChatBot: 
User: winner winner
['menu']
ChatBot: 
User: pathetic food you guys serve
['negative_feedback']
ChatBot: 
User: why why why this much cost
['negative_feedback']
ChatBot: 
User: love your food, keep it up guys
['positive_feedback']
ChatBot: 
User: bored with the regular food, surprise me
['suggest']
ChatBot: 
User: okay, what precautions are you guys taking during covid? I need to some assurance that the food I am eating is safe
['sanitation']
ChatBot: I